In [48]:
# Libraries
from get_data import fetch_data # Module to fetch and process data
import pandas as pd
import numpy as np
import pyarrow.parquet as pq

import datetime

# import config
from sodapy import Socrata
api_key = os.environ.get("SOCRATA_APP_TOKEN")
import os

In [92]:
# Retrieving the data directly from Amazon S3
raw_data = pq.read_pandas('./Data/collisions.parquet').to_pandas()
# Modifying column names
raw_data.columns = raw_data.columns.str.lower().str.replace(' ', '_')
raw_data.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,04/14/2021,5:32,,,NaN,NaN,,BRONX WHITESTONE BRIDGE,,,...,Unspecified,,,,4407480,Sedan,Sedan,,,
1,04/13/2021,21:35,BROOKLYN,11217,40.68358,-73.97617,"(40.68358, -73.97617)",,,620 ATLANTIC AVENUE,...,,,,,4407147,Sedan,,,,
2,04/15/2021,16:15,,,NaN,NaN,,HUTCHINSON RIVER PARKWAY,,,...,,,,,4407665,Station Wagon/Sport Utility Vehicle,,,,
3,04/13/2021,16:00,BROOKLYN,11222,NaN,NaN,,VANDERVORT AVENUE,ANTHONY STREET,,...,Unspecified,,,,4407811,Sedan,,,,
4,04/12/2021,8:25,,,0.00000,0.00000,"(0.0, 0.0)",EDSON AVENUE,,,...,Unspecified,,,,4406885,Station Wagon/Sport Utility Vehicle,Sedan,,,


In [50]:
raw_data.crash_date = pd.DatetimeIndex(raw_data.crash_date)
min_date = raw_data.crash_date.min()
max_date = raw_data.crash_date.max()
print(f'The dataset covers the period between {min_date} and {max_date}')
missing_dates = raw_data.crash_date.isna().sum()
print(f'There are {missing_dates} missing times')

The dataset covers the period between 2012-07-01 00:00:00 and 2022-01-04 00:00:00
There are 0 missing times


In [51]:
# Latitude and longitude coordinates are: 43.000000, -75.000000.
out_of_nyc = raw_data[(raw_data.longitude < -75) | (raw_data.longitude > -73) | (raw_data.latitude > 41) | (raw_data.latitude < 40)]
out_of_nyc.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
4,2021-04-12,8:25,,,0.0,0.0,"(0.0, 0.0)",EDSON AVENUE,,,...,Unspecified,,,,4406885,Station Wagon/Sport Utility Vehicle,Sedan,,,
97,2021-04-15,9:27,,,0.0,0.0,"(0.0, 0.0)",NORWAY AVENUE,,,...,Unspecified,,,,4407538,Station Wagon/Sport Utility Vehicle,Sedan,,,
314,2021-04-06,16:00,,,0.0,0.0,"(0.0, 0.0)",BEDFORD AVENUE,,,...,,,,,4408107,Bus,,,,
427,2021-04-14,15:00,BROOKLYN,11235,0.0,0.0,"(0.0, 0.0)",GERALD COURT,EAST 7 STREET,,...,Unspecified,,,,4408054,Sedan,,,,
657,2021-04-14,11:10,,,0.0,0.0,"(0.0, 0.0)",90 AVENUE,,,...,Unspecified,Unspecified,,,4407319,Box Truck,Sedan,Sedan,,


In [52]:
# Replacing these values as NaN
raw_data.loc[out_of_nyc.index, 'latitude'] = np.nan
raw_data.loc[out_of_nyc.index, 'longitude'] = np.nan
raw_data.loc[out_of_nyc.index, 'location'] = np.nan

In [53]:
raw_data.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2021-04-14,5:32,,,NaN,NaN,,BRONX WHITESTONE BRIDGE,,,...,Unspecified,,,,4407480,Sedan,Sedan,,,
1,2021-04-13,21:35,BROOKLYN,11217,40.68358,-73.97617,"(40.68358, -73.97617)",,,620 ATLANTIC AVENUE,...,,,,,4407147,Sedan,,,,
2,2021-04-15,16:15,,,NaN,NaN,,HUTCHINSON RIVER PARKWAY,,,...,,,,,4407665,Station Wagon/Sport Utility Vehicle,,,,
3,2021-04-13,16:00,BROOKLYN,11222,NaN,NaN,,VANDERVORT AVENUE,ANTHONY STREET,,...,Unspecified,,,,4407811,Sedan,,,,
4,2021-04-12,8:25,,,NaN,NaN,NaN,EDSON AVENUE,,,...,Unspecified,,,,4406885,Station Wagon/Sport Utility Vehicle,Sedan,,,


In [54]:
raw_data[
    (raw_data.longitude < -75) | 
    (raw_data.longitude > -73) | 
    (raw_data.latitude > 41) | 
    (raw_data.latitude < 40)
][['latitude', 'longitude']].shape

(0, 2)

In [55]:
test = raw_data[['number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed']]

In [56]:
test['sum_injured'] = test[['number_of_pedestrians_injured', 'number_of_cyclist_injured', 'number_of_cyclist_killed', 'number_of_motorist_injured']].sum(axis=1)
test

/var/folders/78/5vwtk1hn24jcpgsl685n0blw0000gn/T/ipykernel_70751/3817881941.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['sum_injured'] = test[['number_of_pedestrians_injured', 'number_of_cyclist_injured', 'number_of_cyclist_killed', 'number_of_motorist_injured']].sum(axis=1)


,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,sum_injured
0,0.0,0.0,0,0,0,0,0,0,0
1,1.0,0.0,1,0,0,0,0,0,1
2,0.0,0.0,0,0,0,0,0,0,0
3,0.0,0.0,0,0,0,0,0,0,0
4,0.0,0.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1855784,0.0,0.0,0,0,0,0,0,0,0
1855785,2.0,0.0,0,0,0,0,2,0,2
1855786,0.0,0.0,0,0,0,0,0,0,0
1855787,0.0,0.0,0,0,0,0,0,0,0


In [58]:
test[test['sum_injured'] != test['number_of_persons_injured']]

,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,sum_injured
530,1.0,0.0,0,0,0,0,0,0,0
929,1.0,0.0,0,0,0,0,0,0,0
1212,1.0,0.0,0,0,0,0,0,0,0
1238,1.0,0.0,0,0,0,0,0,0,0
1456,1.0,0.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1813639,0.0,1.0,0,0,0,1,0,0,1
1830866,0.0,1.0,0,0,0,1,0,0,1
1837824,0.0,1.0,0,0,0,1,0,0,1
1845467,0.0,1.0,0,0,0,1,0,0,1


In [8]:
# Filtering for rows with missing values for either persons injured or killed
missing_persons = raw_data[(raw_data['number_of_persons_injured'].isna()) 
        | (raw_data['number_of_persons_killed'].isna())]
missing_persons.shape

(37, 29)

In [28]:
raw_data['number_of_persons_injured'] = raw_data['number_of_pedestrians_injured'] + raw_data['number_of_cyclist_injured'] + raw_data['number_of_motorist_injured']
raw_data['number_of_persons_killed'] = raw_data['number_of_pedestrians_killed'] + raw_data['number_of_cyclist_killed'] + raw_data['number_of_motorist_killed']

In [33]:
total_number_of_persons_killed = raw_data.number_of_persons_killed.sum()
total_number_of_persons_killed

2461

In [32]:
sum(raw_data['number_of_pedestrians_killed'] + raw_data['number_of_cyclist_killed'] + raw_data['number_of_motorist_killed'])

2461

In [24]:
raw_data['number_of_persons_injured'] = raw_data[['number_of_pedestrians_injured', 'number_of_cyclist_injured','number_of_motorist_injured']].sum(axis=1)
raw_data['number_of_persons_killed'] = raw_data[['number_of_pedestrians_injured', 'number_of_cyclist_injured','number_of_motorist_injured']].sum(axis=1)
# raw_data.loc[raw_data, 'number_of_persons_killed'] = missing_persons[['number_of_pedestrians_injured', 'number_of_cyclist_injured', 'number_of_motorist_injured']].sum(axis=1)

In [14]:
raw_data.loc[missing_persons.index, 'number_of_persons_injured'] = missing_persons[['number_of_pedestrians_injured', 'number_of_cyclist_injured','number_of_motorist_injured']].sum(axis=1)
raw_data.loc[missing_persons.index, 'number_of_persons_killed'] = missing_persons[['number_of_pedestrians_injured', 'number_of_cyclist_injured', 'number_of_motorist_injured']].sum(axis=1)

In [25]:
raw_data[['number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed']].isna().sum()

number_of_persons_injured        0
number_of_persons_killed         0
number_of_pedestrians_injured    0
number_of_pedestrians_killed     0
number_of_cyclist_injured        0
number_of_cyclist_killed         0
number_of_motorist_injured       0
number_of_motorist_killed        0
dtype: int64

In [27]:
np.where(raw_data.number_of_persons_injured != raw_data[['number_of_pedestrians_injured', 'number_of_cyclist_injured','number_of_motorist_injured']].sum(axis=1))

(array([], dtype=int64),)

In [121]:
raw_data

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2021-04-14,5:32,,,NaN,NaN,,BRONX WHITESTONE BRIDGE,,,...,Unspecified,,,,4407480,Sedan,Sedan,,,
1,2021-04-13,21:35,BROOKLYN,11217,40.683580,-73.976170,"(40.68358, -73.97617)",,,620 ATLANTIC AVENUE,...,,,,,4407147,Sedan,,,,
2,2021-04-15,16:15,,,NaN,NaN,,HUTCHINSON RIVER PARKWAY,,,...,,,,,4407665,Station Wagon/Sport Utility Vehicle,,,,
3,2021-04-13,16:00,BROOKLYN,11222,NaN,NaN,,VANDERVORT AVENUE,ANTHONY STREET,,...,Unspecified,,,,4407811,Sedan,,,,
4,2021-04-12,8:25,,,NaN,NaN,NaN,EDSON AVENUE,,,...,Unspecified,,,,4406885,Station Wagon/Sport Utility Vehicle,Sedan,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855784,2012-07-06,15:09,MANHATTAN,10035,40.801235,-73.941815,"(40.8012354, -73.9418153)",EAST 119 STREET,PARK AVENUE,,...,Unspecified,,,,59654,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,,,
1855785,2012-07-03,17:30,QUEENS,11102,40.774711,-73.933386,"(40.7747112, -73.9333863)",27 AVENUE,4 STREET,,...,Unspecified,,,,272592,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,,,
1855786,2012-07-01,15:30,BROOKLYN,11236,40.645032,-73.919978,"(40.6450318, -73.9199775)",RALPH AVENUE,CLARENDON ROAD,,...,Unspecified,,,,135041,SMALL COM VEH(4 TIRES),PASSENGER VEHICLE,,,
1855787,2012-07-08,18:30,,,40.786122,-73.804078,"(40.7861217, -73.8040782)",,,,...,Unspecified,,,,3055617,PASSENGER VEHICLE,PASSENGER VEHICLE,,,
